In [32]:
!pip install binarytree

In [50]:
from ast import Expression
import re
from binarytree import Node
from IPython.display import display
import ipywidgets as widgets



reserved_words = ['if', 'then', 'else', 'end', 'repeat', 'until', 'read', 'write']
special_symbols = ['+', '-', '*', '/', '=', '<', '(', ')', ';', ':=', '{', '}']


identifier_pattern = r'[a-zA-Z_][a-zA-Z0-9_]*'
float_pattern = r'\b\d+\.\d+\b'
int_pattern = r'\b\d+\b'
comment_pattern = r'%%[a-zA-Z]*'


def is_reserved_word(token):
    return token in reserved_words

def is_special_symbol(token):
    return token in special_symbols

def is_identifier(token):
    return re.fullmatch(identifier_pattern, token)

def is_float(token):
    return re.fullmatch(float_pattern, token)

def is_int(token):
    return re.fullmatch(int_pattern, token)

def is_comment(token):
    return re.fullmatch(comment_pattern, token)


def preprocess_equation(equation):
    equation = equation.replace('pi', '3.14')
    equation = equation.replace('PI', '3.14')
    equation = re.sub(r'(\d)(\()', r'\1 * \2', equation)

    tokens = re.findall(r'[\w.]+|:=|[+\-*/=<>();{}]', equation)
    unique_identifiers = {}
    new_expression = []
    id_counter = 1

    for token in tokens:
        if is_reserved_word(token):
            new_expression.append(token)
        elif is_special_symbol(token):
            new_expression.append(token)
        elif is_float(token) or is_int(token):
            new_expression.append(token)
        elif is_identifier(token):
            if token not in unique_identifiers:
                id_name = f'id{id_counter}'
                unique_identifiers[token] = id_name
                id_counter += 1
            new_expression.append(unique_identifiers[token])
        elif is_comment(token):
            continue
        else:
            raise ValueError(f"Unknown token: {token}")

    return " ".join(new_expression), unique_identifiers

#  using Shunting Yard algorithm
def infix_to_postfix(expression):
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2, '=': 0}
    stack = []
    postfix = []
    tokens = expression.split()

    for token in tokens:
        if is_float(token) or is_int(token) or is_identifier(token):
            postfix.append(token)
        elif token in precedence:
            while stack and precedence.get(stack[-1], -1) >= precedence[token]:
                postfix.append(stack.pop())
            stack.append(token)
        elif token == '(':
            stack.append(token)
        elif token == ')':
            while stack and stack[-1] != '(':
                postfix.append(stack.pop())
            stack.pop()

    while stack:
        postfix.append(stack.pop())

    return postfix


def build_syntax_tree(postfix):
    stack = []
    for token in postfix:
        if is_float(token) or is_int(token) or is_identifier(token):
            stack.append(Node(token))
        else:
            right = stack.pop()
            left = stack.pop()
            node = Node(token)
            node.left = left
            node.right = right
            stack.append(node)
    return stack.pop()


def build_semantic_tree(postfix):
    stack = []
    has_float = any(is_float(token) for token in postfix)

    for token in postfix:
        if is_float(token) or is_int(token) or is_identifier(token):
            node = Node(token)

            if has_float and is_int(token):
                inttofloat_node = Node("inttofloat")
                inttofloat_node.left = node
                stack.append(inttofloat_node)
            else:
                stack.append(node)
        else:
            right = stack.pop()
            left = stack.pop()
            operator_node = Node(token)
            operator_node.left = left
            operator_node.right = right
            stack.append(operator_node)

    return stack.pop()
def generate_intermediate_code(semantic_tree):
    temp_counter = 1
    intermediate_code = []

    def traverse(node):
        nonlocal temp_counter
        if node is None:
            return None


        if node.value == "inttofloat":
            operand = traverse(node.left)
            temp_var = f"Temp{temp_counter}"
            temp_counter += 1
            intermediate_code.append(f"{temp_var} = inttofloat({operand})")
            return temp_var

        if node.value == "=":
             right = traverse(node.right)
             intermediate_code.append(f"id1 = {right}")
             return right


        if node.value in ['+', '-', '*', '/', '<', '>']:
            left = traverse(node.left)
            right = traverse(node.right)

            temp_var = f"Temp{temp_counter}"
            temp_counter += 1
            intermediate_code.append(f"{temp_var} = {left} {node.value} {right}")
            return temp_var

        return node.value

    root_result = traverse(semantic_tree)


    return intermediate_code

def code_optimizer(intermediate_code):
    optimized_code = []
    temp_map = {}
    new_temp_counter = 1

    for i, line in enumerate(intermediate_code):
        if "inttofloat" in line:
            temp_var, expression = line.split(" = ")
            number = expression.replace("inttofloat(", "").replace(")", "").strip()
            temp_map[temp_var] = number
        else:
            for old_temp, optimized_value in temp_map.items():
                line = line.replace(old_temp, optimized_value)

            if "Temp" in line:
                temp_var, operation = line.split(" = ", 1)
                if i == len(intermediate_code) - 1:
                    line = f"id1 = {operation.strip()}"
                else:
                    new_temp_var = f"Temp{new_temp_counter}"
                    temp_map[temp_var.strip()] = new_temp_var
                    line = f"{new_temp_var} = {operation.strip()}"
                    new_temp_counter += 1

            optimized_code.append(line)

    return optimized_code


def generate_code_generator(optimized_code):
    # To hold the assembly-like code
    assembly_code = []

    reg1 = "r1"
    reg2 = "r2"

    for line in optimized_code:
        line = line.strip()
        if not line:
            continue

        if "=" in line:
            lhs, rhs = line.split("=", 1)
            lhs = lhs.strip()
            rhs = rhs.strip()

            # Check for arithmetic operations
            if any(op in rhs for op in ["+", "-", "*", "/"]):
                tokens = re.split(r'(\+|\-|\*|/)', rhs)

                # Handle the first operand
                first_operand = tokens[0].strip()
                if not first_operand.isdigit():
                 assembly_code.append(f"ldf {reg2}, {first_operand}")

                # Process the remaining tokens
                current_register = reg1
                for i in range(1, len(tokens), 2):
                    operator = tokens[i]
                    operand = tokens[i + 1].strip()



                    # Perform the operation
                    if operator == "+":
                        assembly_code.append(f"addf {current_register}, {current_register}")
                    elif operator == "-":
                        assembly_code.append(f"subf {current_register}, {current_register}")
                    elif operator == "*":
                        assembly_code.append(f"mulf {current_register}, {current_register}")
                    elif operator == "/":
                        assembly_code.append(f"divf {current_register}, {current_register}")

                # Store the final result in the left-hand side variable
                if lhs.startswith("id"):
                    assembly_code.append(f"stf {lhs}, {current_register}")

            else:
                # Handle simple assignments
                assembly_code.append(f"ldf {reg1}, {rhs}")
                assembly_code.append(f"stf {lhs}, {reg1}")

    return assembly_code




def main_gui():

    input_box = widgets.Text(
        description="Input:",
        placeholder="Enter your equation",
        layout=widgets.Layout(width="70%")
    )
    input_choice = widgets.RadioButtons(
        options=["Math Form", "Source Form"],
        description="Form:",
        style={"button_color": "lightgray"},
        layout=widgets.Layout(width="50%")
    )
    submit_button = widgets.Button(description="Submit")


    lexical_output = widgets.Output()
    syntax_tree_output = widgets.Output()
    semantic_tree_output = widgets.Output()
    intermediate_code_output = widgets.Output()
    optimized_code_output = widgets.Output()
    machine_code_output = widgets.Output()


    accordion = widgets.Accordion(children=[
        lexical_output,
        syntax_tree_output,
        semantic_tree_output,
        intermediate_code_output,
        optimized_code_output,
        machine_code_output
    ])
    accordion.set_title(0, "Lexical Analysis")
    accordion.set_title(1, "Syntax Tree")
    accordion.set_title(2, "Semantic Tree")
    accordion.set_title(3, "Intermediate Code")
    accordion.set_title(4, "Optimized Code")
    accordion.set_title(5, "Machine Code")


    def on_submit(_):
        user_input = input_box.value
        form_choice = input_choice.value

        if not user_input.strip():
            with lexical_output:
                print("Error: Input cannot be empty.")
            return

        try:

            if form_choice == "Math Form":
                modified_expression, lexical_result = preprocess_equation(user_input)
            elif form_choice == "Source Form":
                if 'pi' in user_input.lower():
                    with lexical_output:
                        print("Lexical Error: 'pi' is not allowed in Source Form.")
                    return
                    modified_expression, lexical_result = preprocess_equation(user_input)


            lexical_output.clear_output()
            with lexical_output:
                print("lexical analyzer:", modified_expression)
                print("Identifiers Mapping:", lexical_result)

            postfix_expression = infix_to_postfix(modified_expression)

            syntax_tree = build_syntax_tree(postfix_expression)
            syntax_tree_output.clear_output()
            with syntax_tree_output:
                print("Postfix Expression:", ' '.join(postfix_expression))
                print("\nSyntax Tree:")
                print(syntax_tree)

            semantic_tree = build_semantic_tree(postfix_expression)
            semantic_tree_output.clear_output()
            with semantic_tree_output:
                print("\nSemantic Tree")
                print(semantic_tree)

       # Call the function to generate intermediate code
            intermediate_code = generate_intermediate_code(semantic_tree)

# Clear the output to avoid any cached results
            intermediate_code_output.clear_output(wait=True)

# Print the intermediate code
            with intermediate_code_output:
             print("\nIntermediate Code:")
             for line in intermediate_code:
              print(line)

            optimized_code = code_optimizer(intermediate_code)
            optimized_code_output.clear_output()
            with optimized_code_output:
                print("\nOptimized Code:")
                for line in optimized_code:
                    print(line)

            machine_code = generate_code_generator(optimized_code)
            machine_code_output.clear_output()
            with machine_code_output:
                print("\n Code generator")
                for line in machine_code:
                    print(line)

        except Exception as e:
            lexical_output.clear_output()
            with lexical_output:
                print(f"Error: {str(e)}")

    submit_button.on_click(on_submit)


    display(widgets.VBox([
        widgets.HBox([input_box, input_choice]),
        submit_button,
        accordion
    ]))


main_gui()
